# CCS potential with LAMMPS  for ZnO

In this example, we will run CCS potentials with LAMMPS for multicomponent systems. ZnO  CCS potential is used for this illustration. Note that the exponential head part in **CCS_params.json** is ignored when LAMMPS spline tables are created. The exponential part is avoided because LAMMPS performs a pre-interpolation on the tabular data, and often pre-interpolation done with exponential head gives worse results. In general, a higher resolution should be used while generating spline table for LAMMPS. By default, the gridsize used for CCS is divided by 10.

**Note:** 
* Install scikit-learn if it's not installed yet; 
https://scikit-learn.org/stable/install.html

In [1]:
# Generates CCS.table file readable by LAMMPS
from ccs_fit.scripts.jsonTotable import asecalcTotable
tags=asecalcTotable("CCS_params.json",scale=10)  # Controls the resolution of the gridsize; gridsize=dr/scale.
tags

In [2]:
!head CCS.table

In [3]:
import json
import copy
import matplotlib.pyplot as plt

from ase import Atoms
from ase.calculators.lammpsrun import LAMMPS
from ccs_fit.ase_calculator.ccs_ase_calculator import CCS
from ase.io import read,write
from ase.io.trajectory import Trajectory
from ase.optimize import BFGS
from sklearn.metrics import mean_squared_error
import numpy as np


with open ('CCS_params.json','r') as f:
    CCS_params=json.load(f)


# Parameters for LAMMPS 
parameters = {
              'pair_style': 'table spline 1000',   # The number of elements chosen taken for pre-interpolation
              'pair_coeff': ['1 1  CCS.table O-O 5.9999990640749274', # Specify Rcut for each pair
                             '1 2  CCS.table O-Zn 5.9999991089416',
                             '2 2  CCS.table Zn-Zn 5.999995708965134'],
#              'pair_write': ['1 2 500 r  1.2999991089415996 5.9999991089416 table.txt table']
              'command': '/usr/bin/lmp'   # Remove or change to your local lammps executable
}



# Single point evaluation

In [4]:
struct=read('TiO2.poscar')

chem_symbs = struct.get_chemical_symbols()
chem_symbs[0] = 'Zn'
chem_symbs[1] = 'Zn'

struct.set_chemical_symbols(chem_symbs)

In [5]:
from pymatgen.core import Structure

struc_pmg = Structure.from_file("TiO2.POSCAR")

print(dir(struc_pmg))

print(struc_pmg.get_space_group_info())

In [6]:
# struct=read('POSCAR')
lammps = LAMMPS(parameters=parameters,keep_tmp_files=False)
ccs= CCS(CCS_params)


struct_ase=copy.deepcopy(struct)

struct.calc = lammps
print("Energy from LAMMPS  pre optmization: ", struct.get_potential_energy())
struct_ase.calc=ccs
print("Energy from CCS calculator pre optmization:", struct_ase.get_potential_energy())

print ("MSE on energy between ase calc and LAMMPS: {}".format((struct_ase.get_potential_energy()-struct.get_potential_energy())**2))
print ("MSE on forces between ase calc and LAMMPS: {}".format(mean_squared_error(struct.get_forces(),struct_ase.get_forces())))




# Optimization

In [7]:
traj = Trajectory('LAMMPS.traj', 'w', struct)
dyn=BFGS(struct)
dyn.attach(traj)
dyn.run(fmax=0.005)
print("Energy from LAMMPS after optimization: ", struct.get_potential_energy())


traj = Trajectory('CCS.traj', 'w', struct_ase)
dyn=BFGS(struct_ase)
dyn.attach(traj)
dyn.run(fmax=0.005)
print("Energy from CCS calculator after optimization:", struct_ase.get_potential_energy())

print ("MSE on energy between ase calc and LAMMPS: {}".format((struct_ase.get_potential_energy()-struct.get_potential_energy())**2))
print ("MSE on forces between ase calc and LAMMPS: {}".format(mean_squared_error(struct.get_forces(),struct_ase.get_forces())))



**Note** : 
* Onebody energy contributons in CCS_params file was set to 0 for easy comparison

### Adding cell optimisation

In [8]:
from ase.constraints import StrainFilter
from ase.io import read

In [13]:
opt_struct = read('LAMMPS.traj')
opt_struct.calc = lammps

print(opt_struct.get_stress())

orig_cell = opt_struct.cell.copy()

sf = StrainFilter(opt_struct)
opt = BFGS(sf, trajectory='LAMMPS_strain.traj')
opt.run(fmax=1e-4)
alat = opt_struct.cell[0][1] - opt_struct.cell[0][0]

LAMMPS_strain_opt = read('LAMMPS_strain.traj')
LAMMPS_strain_opt_cell = LAMMPS_strain_opt.cell

old_vol = np.prod(np.diag(orig_cell))
new_vol = np.prod(np.diag(LAMMPS_strain_opt_cell))

print("Volume change of {} %, as a result of the following lattice parameter changes:".format((new_vol-old_vol)/old_vol*100))
print([np.diag(LAMMPS_strain_opt_cell-orig_cell)[i]/np.diag(orig_cell)[i]*100 for i in range(3)])

In [14]:
from ase.io import write, read

opt_struct = read('CCS.traj')
opt_struct.calc = ccs

print(opt_struct.get_stress())

orig_cell = opt_struct.cell.copy()

sf = StrainFilter(opt_struct)
opt = BFGS(sf, trajectory='CCS_strain.traj')
opt.run(fmax=1e-4)
alat = opt_struct.cell[0][1] - opt_struct.cell[0][0]

CCS_strain_opt = read('CCS_strain.traj')
CCS_strain_opt_cell = CCS_strain_opt.cell

old_vol = np.prod(np.diag(orig_cell))
new_vol = np.prod(np.diag(CCS_strain_opt_cell))

write("TiO2_opt_ccs.POSCAR", CCS_strain_opt)

print("Volume change of {} %, as a result of the following lattice parameter changes:".format((new_vol-old_vol)/old_vol*100))
print([np.diag(CCS_strain_opt_cell-orig_cell)[i]/np.diag(orig_cell)[i]*100 for i in range(3)])

print(np.diag(CCS_strain_opt_cell))

In [11]:
!ase gui LAMMPS.traj

In [12]:
!ase gui LAMMPS_strain.traj

In [1]:
from ccs_fit.scripts.ccs_export_FF import write_FF

write_FF("CCS_params.json")